In [ ]:
import pandas as pd
import numpy as np
df_policy = pd.read_csv('policy_claim/policy_0702.csv')
df_claim = pd.read_csv('policy_claim/claim_0702.csv')
df_training = pd.read_csv('training-set.csv')
df_testing = pd.read_csv('testing-set.csv')

## 0. Read Files

### policy

In [ ]:
print(df_policy.shape) # 41 items  (1747942, 41)
print('unique:',df_policy['Policy_Number'].nunique()) #351273 unique Policy Number
pd.set_option('display.max_columns', None)
df_policy.head()   

In [ ]:
print(df_policy.columns)

In [ ]:
### examine Drivers
driver = df_policy[['ibirth' , 'dbirth']]
print(driver.shape[0])
driver.head()

In [ ]:
condition1 = driver['ibirth'].isnull()    # 18.2 %
condition2 = driver['dbirth'].isnull()    # 14.8 %

condition3 = driver['ibirth'].notnull()   
condition4 = driver['dbirth'].notnull()   


driver_0N= driver[condition3&condition4]  # both have values  (81.5%)
driver_1N= driver[condition1^condition2]  # missing one (3.9%)
driver_2N= driver[condition1|condition2]  # missing both (18.4%)

print(driver_0N.shape[0]/driver.shape[0])
print(driver_1N.shape[0]/driver.shape[0]) 
print(driver_2N.shape[0]/driver.shape[0])

In [ ]:
# check indiff. in 0N

indiff = driver_0N[driver_0N['ibirth']!=driver_0N['dbirth']]
print(indiff.shape[0])
indiff.sample(10)

In [ ]:
### dbirth has some weird date
### check non-null data & sort
print(driver['dbirth'].notnull().sum())
import datetime
a = df_policy[driver['dbirth'].notnull()]['dbirth']
b = sorted(a, key=lambda x: datetime.datetime.strptime(x, '%m/%Y'), reverse = True)
b

In [ ]:
replace_date = {
    '09/2658',
 '09/2658',
 '09/2658',
 '09/2658',
 '10/2622',
 '10/2622',
 '10/2622',
 '10/2622',
 '02/2611',
 '02/2611',
 '02/2611',
 '02/2611',
 '02/2611',
 '02/2611',
 '11/2602',
 '11/2602',
 '11/2602',
 '11/2602',
 '11/2602',
 '11/2602',
 '11/2602',
 '11/2602',
 '10/2582',
 '10/2582',
 '10/2582',
 '10/2582',
 '10/2581',
 '10/2581',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '08/2569',
 '08/2569',
 '08/2569',
 '08/2569',
 '08/2569',
 '10/2521',
 '11/2512',
 '11/2512',
 '11/2512',
 '11/2512',
 '11/2512',
 '01/2512',
 '01/2512',
 '01/2512',
 '01/2512',
 '01/2512',
 '10/2472',
 '10/2472',
 '08/2471',
 '08/2471',
 '08/2471',
 '08/2471',
 '08/2471',
 '11/2464',
 '11/2464',
 '11/2464',
 '03/2464',
 '03/2464',
 '12/2453',
 '12/2453',
 '12/2453',
 '12/2453',
 '01/2442',
 '01/2442',
 '01/2442',
 '01/2442',
 '01/2442',
 '01/2442',
 '07/2280',
 '07/2280',
 '07/2280',
 '07/2280',
 '07/2280',
 '07/2280',
 '07/2280',
 '07/2280',
 '02/2152',
 '02/2152',
 '02/2152',
 '02/2152',
 '02/2152',
 '02/2152',
 '02/2152',
 '02/2152',
 '02/2152'}
len(replace_date)

In [ ]:
replace_date_set = set(replace_date)
len(replace_date_set)

In [ ]:
date_switch = df_policy.apply(lambda row: True if row['dbirth'] in replace_date_set else False, axis = 1)
df_policy_selected = df_policy[date_switch]
print(df_policy_selected.shape[0])
df_policy_selected.head()

In [ ]:
selected_idx = df_policy_selected.index.values
selected_idx

In [ ]:
df_policy_selected_date = df_policy[['ibirth', 'dbirth','fassured']].loc[df_policy_selected.index]
# "1:國內自然人 2:國內法人 3:國外自然人 4:國外法人 5:自然人ID有誤 6:法人ID有誤"
df1 = df_policy_selected_date[df_policy_selected_date['fassured']==1]
df2 = df_policy_selected_date[df_policy_selected_date['fassured']==2]
print(df1.shape[0])
print(df2.shape[0])

In [ ]:
df2.head() ## replace as NaN  (as ibirth)

In [ ]:
df1.head() ## replace as ibirth

In [ ]:
df_policy['ID']=df_policy.index   # create ID column for ref.
df_policy.head()

In [ ]:
### replace weird date as ibirth
df_policy['dbirth_new'] = df_policy.apply(lambda row: row['ibirth'] if row['ID'] in selected_idx 
                                          else row['dbirth'], axis = 1)

In [ ]:
df_policy[['ibirth', 'dbirth', 'fassured','dbirth_new']].loc[selected_idx].sample(10)

In [ ]:
df_policy['dbirth'] = df_policy['dbirth_new']
df_policy = df_policy.drop(['dbirth_new','ID'], axis =1)
df_policy.head()

In [ ]:
print(df_policy.shape)

### Claim

In [ ]:
print(df_claim.shape)  # 20 items
print('unique:',df_claim['Policy_Number'].nunique()) #35895 unique Policy Number
pd.set_option('display.max_columns', None)  ## show all columns
df_claim.head()

In [ ]:
print(df_claim.columns)

### training & testing

In [ ]:
print(df_training.shape)
print('unique:',df_training['Policy_Number'].nunique()) #210763 unique Policy Number
df_training.head()

In [ ]:
df_training[df_training['Next_Premium']==100]  # if Next_Premium = 100 -> testing_set

In [ ]:
print(df_testing.shape)
print('unique:',df_testing['Policy_Number'].nunique()) #140510 unique Policy Number
df_testing.head()

In [ ]:
a=df_training['Policy_Number'].nunique()/df_policy['Policy_Number'].nunique()*100
b=df_testing['Policy_Number'].nunique()/df_policy['Policy_Number'].nunique()*100
print('training set (pct)  %f '%a)
print('testing set (pct)  %f '%b)

## 1. Combine policy & claim & training & testing

In [ ]:
df_all= df_policy.merge(df_claim, on=df_policy.columns[0], how='left', suffixes=('','_Claim'))
print(df_all.columns)
print(df_all.shape)   # 60 items (overlap on Policy_Number only)

In [ ]:
df_all= df_all.merge(df_training, on=df_policy.columns[0], how='left')
df_all= df_all.merge(df_testing, on=df_policy.columns[0], how='left')
df_all['Next_Premium'] = df_all.apply(
    lambda row: row['Next_Premium_y'] if row['Next_Premium_y']==100 else row['Next_Premium_x'], axis = 1)
df_all=df_all.drop(['Next_Premium_x','Next_Premium_y'], axis =1)
print(df_all.shape) 
print(df_all.columns)

In [ ]:
df_all.info()

### 1.1 check missing data

In [ ]:
# % percentage
df_all.isnull().sum()/df_all.shape[0]*100   # about 22.7 declaire claim

In [ ]:
### df_all.nunique()

# '''
# Policy_Number                                 351273  Encode
# Insured's_ID                                  299999  Encode
# Prior_Policy_Number                           281678  Encode
# Cancellation                                       2  Dummy
# Vehicle_identifier                            347882  Encode
# Vehicle_Make_and_Model1                          136  Encode (too many...)
# Vehicle_Make_and_Model2                         8112  Encode
# Manafactured_Year_and_Month                       49  Datetime
# Engine_Displacement_(Cubic_Centimeter)           768  no Change
# Imported_or_Domestic_Car                          10  Dummy
# Coding_of_Vehicle_Branding_&_Type               7405  Encode
# qpt                                               19  Dummy
# fpt                                                1  Dummy
# Main_Insurance_Coverage_Group                      3  Dummy
# Insurance_Coverage                                60  Dummy
# Insured_Amount1                                  111  no Change
# Insured_Amount2                                   89  no Change
# Insured_Amount3                                 3071  no Change
# Coverage_Deductible_if_applied                    53  no Change
# Premium                                        18238  no Change
# Replacement_cost_of_insured_vehicle             2947  no Change
# Distribution_Channel                             875  Encode (too many...)
# Multiple_Products_with_TmNewa_(Yes_or_No?)       179  no Change (dummy??)
# lia_class                                         21  no Change (dummy??)
# plia_acc                                          21  no Change (dummy??)
# pdmg_acc                                          13  no Change (dummy??)
# fassured                                           4  Dummy
# ibirth                                           918  Datetime
# fsex                                               4  Dummy
# fmarriage                                          4  Dummy 
# aassured_zip                                    1722  Encode (too many...)                                                                       ...  
# iply_area                                         22  Dummy
# dbirth                                           945  Datetime
# fequipment1                                        2  Dummy
# fequipment2                                        2  Dummy
# fequipment3                                        2  Dummy
# fequipment4                                        1  Dummy
# fequipment5                                        2  Dummy
# fequipment6                                        1  Dummy
# fequipment9                                        2  Dummy 
# nequipment9                                        6  Dummy
# ============ Claim  ========
# Claim_Number                                   41137  Encode
# Nature_of_the_claim                                2  Dummy
# Driver's_Gender                                    2  Dummy
# Driver's_Relationship_with_Insured                 7  Dummy
# DOB_of_Driver                                    799  Datetime
# Marital_Status_of_Driver                           2  Dummy
# Accident_Date                                     25  Datetime
# Cause_of_Loss                                     17  Dummy
# Paid_Loss_Amount                               17229  no Change 
# paid_Expenses_Amount                             799  no Change
# Salvage_or_Subrogation?                         2056  no Change
# Coverage                                          48  Encode (same as Insurance_Coverage!)
# Vehicle_identifier_Claim                       35714  Encode (same as Vehicle_identifier if applicable!)
# At_Fault?                                         23  no Change (dummy??)
# Claim_Status_(close,_open,_reopen_etc)             2  Dummy
# Deductible                                        67  no Change 
# Accident_area                                     22  Dummy  (same as aassured_zip?)
# number_of_claimants                               14  no Change (dummy??)
# Accident_Time                                     39  DateTime
# Next_Premium                                   22817  no Change
# '''

In [ ]:
## fill na as 0 (Check explaination document, those 0 doesn't mean anything)

df= df_all
cols = ['Prior_Policy_Number','Vehicle_identifier','fsex','fmarriage','Claim_Number','Nature_of_the_claim','Policy_Number',
        "Driver's_Gender","Driver's_Relationship_with_Insured",'Marital_Status_of_Driver',
        'Cause_of_Loss','number_of_claimants']
for col in cols:
    df[col]=df[col].fillna(0)
    print(col, df[col].isnull().sum())


In [ ]:
df_all = df

In [ ]:
## fill na as -1 ( becasue 0 means something else)

df= df_all
cols = ['Paid_Loss_Amount','paid_Expenses_Amount','Salvage_or_Subrogation?', 'At_Fault?', 
        'Claim_Status_(close,_open,_reopen_etc)', 'Deductible']
for col in cols:
    df[col]=df[col].fillna(-1)
    print(col, df[col].isnull().sum())



In [ ]:
df_all = df

In [ ]:
##### Verify and drop 
df = df_all
### verify if 'Vehicle_identifier' = 'Vehicle_identifier_Claim'  (YES)
df_temp=df[['Policy_Number','Vehicle_identifier','Vehicle_identifier_Claim']]
df_temp2 = df_temp[df_temp['Vehicle_identifier_Claim'].notnull()] 
print(df_temp2.shape)
df_temp2[df_temp2['Vehicle_identifier']==df_temp2['Vehicle_identifier_Claim']]
print(df_temp2.shape)     

### verify if 'Insurance_Coverage'=='Coverage'    (YES)
df_temp=df[['Policy_Number','Insurance_Coverage','Coverage']]
df_temp2 = df_temp[df_temp['Coverage'].notnull()] 
print(df_temp2.shape)
df_temp2.head()
df_temp2[df_temp2['Insurance_Coverage']==df_temp2['Coverage']]
print(df_temp2.shape)  # 'Insurance_Coverage'=='Coverage' if there is value

### verify if 'Accident_area'=='iply_area'      (YES)
df_temp=df[['Policy_Number','Accident_area','iply_area']]
df_temp2 = df_temp[df_temp['Accident_area'].notnull()] 
print(df_temp2.shape)
df_temp2.head()
df_temp2[df_temp2['iply_area']==df_temp2['Accident_area']]
print(df_temp2.shape)  # 'Accident_area'=='iply_area'' if there is value

In [ ]:
## drop 'Coverage' & 'Vehicle_identifier_Claim' & 'Accident_area
df= df_all
df = df.drop(['Coverage','Vehicle_identifier_Claim','Accident_area'], axis=1)
df.shape

In [ ]:
df_all = df

In [ ]:
### fill NaN as (today)

# df= df_all
# cols = ['ibirth','dbirth','DOB_of_Driver','Accident_Date','Accident_Time']
# for col in cols:
#     df[col]=df[col].fillna(0)
#     print(col, df[col].isnull().sum())

In [ ]:
df_all = df

In [ ]:
# df_all.isnull().sum()   

In [ ]:
df = df_all
df[df.columns[41:]].nunique()

### 1.2 Label Encoder: convert anon id to No.

In [ ]:
pd.set_option('display.max_columns', None)  ## show all columns
print(df_all.shape)
df_all.head()

In [ ]:
'''
encoded columns:

Policy_Number	保單號碼
Insured's_ID	被保險人代號(替代值)
Prior_Policy_Number	前保單號
Vehicle_identifier	車牌號碼                               ## same as vehicle_identifier
Vehicle_Make_and_Model1	廠牌名稱一                   
Vehicle_Make_and_Model2	廠牌名稱二
Coding_of_Vehicle_Branding_&_Type	廠牌車型代號
Distribution_Channel	主通路代號
aassured_zip	郵遞區號                            
Claim_Number	賠案號碼
Vehicle_identifier_Claim	車牌                           ## same as vehicle_identifier (already drop)

'''

In [ ]:
df = df_all

## do not include:  Vehicle_identifier_Claim	車牌  (map to Vehicle_identifier)
cols = ('Policy_Number',"Insured's_ID",'Prior_Policy_Number','Vehicle_identifier', 'Vehicle_Make_and_Model1',
       'Vehicle_Make_and_Model2','Coding_of_Vehicle_Branding_&_Type','Distribution_Channel','aassured_zip',
        'Distribution_Channel','aassured_zip','Claim_Number')


# process columns, apply LabelEncoder to categorical features
from sklearn.preprocessing import LabelEncoder
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(df[c].values)) 
    df[c] = lbl.transform(list(df[c].values))

# shape        
print('Shape of data: {}'.format(df.shape))
df.head()

In [ ]:
### Already dropped
### encode 'Vehicle_identifier_Claim' (NaN: NaN, not NaN: 'Vehicle_identifier' )
# df = df_all
# condition1 = df['Vehicle_identifier_Claim'].isnull()
# condition2 = df['Vehicle_identifier_Claim'].notnull()

# df['Vehicle_identifier_Claim'][condition1]=df['Vehicle_identifier_Claim']
# df['Vehicle_identifier_Claim'][condition2]=df['Vehicle_identifier']

#df[['Policy_Number','Vehicle_identifier','Vehicle_identifier_Claim']].sample(5)

### 1.3 Get dummy

In [ ]:

'''
dummy columns:

Cancellation                   非繼承自前保單	類別	Y: 是 、空白: 否(N)
Imported_or_Domestic_Car       車系代號	類別	參照：車系代號Sheet
qpt                            乘載數量	數值
fpt                            乘載單位	類別	P:載客 T:噸
Main_Insurance_Coverage_Group  險種分類	類別	參照:險種分類及自負額說明Sheet
Insurance_Coverage    ####     險種代碼	類別	參照:險種分類及自負額說明Sheet
fassured                       被保險人性質	類別	"1:國內自然人 2:國內法人 3:國外自然人 4:國外法人 5:自然人ID有誤 6:法人ID有誤"
fsex                           性別	類別	1:男 2:女 空白:法人(0)
fmarriage                      婚姻狀況	類別	1:已婚 2:未婚
iply_area             ####     承保地區代號(分公司)	類別
fequipment1                    配備-音響註記	類別	1:是 0:否
fequipment2                    配備-車箱註記	類別	1:是 0:否
fequipment3                    配備-冷凍機組註記	類別	1:是 0:否
fequipment4                    配備-車框、車斗註記	類別	1:是 0:否
fequipment5                    配備-昇降系統註記	類別	1:是 0:否
fequipment6                    配備-電動車電池註記	類別	1:是 0:否
fequipment9                    配備-其他	類別	1:是 0:否
nequipment9                    配備-其他說明	類別	1:是 0:否
Nature_of_the_claim            賠案性質	類別	1.賠付 2.追償
Driver's_Gender                肇事駕駛性別	類別	1:男 2:女 空白:法人
Driver's_Relationship_with_Insured 與被保險人關係	類別	"1:被保險人本人 2:被保險人親友 3:被保險人員工 4:租用被保險車輛 5:其他, 6:配偶, 7:子女"
Marital_Status_of_Driver       肇事駕駛婚姻	類別	1:已婚 2:未婚
Cause_of_Loss                  出險原因	類別
Coverage             ###       險種代號	類別  same as insurance_coverage (Already Dropped)
Accident_area        ####      出險地區	類別  same as iply (AlreadyDrop)
'''

In [ ]:
## Label cancellation from {'Y', ' '}-> {'Y', 'N'} get dummy later
df= df_all
print(set(df['Cancellation']))
input1 = ' '
replace1 = 'N'
df['Cancellation']=df['Cancellation'].replace(input1, replace1) 
df.head()

In [ ]:
df_all = df

In [ ]:
### Getting dummy categorical features:

In [ ]:
df = df_all
segment=['Cancellation','Imported_or_Domestic_Car','qpt','fpt','Main_Insurance_Coverage_Group','Insurance_Coverage',
         'fassured', 'fsex','fmarriage','iply_area', 'fequipment1','fequipment2','fequipment3','fequipment4',
         'fequipment5','fequipment6','fequipment9','nequipment9','Nature_of_the_claim',"Driver's_Gender",
         "Driver's_Relationship_with_Insured", 'Marital_Status_of_Driver', 'Cause_of_Loss']
df_dummy = pd.get_dummies(df, columns=segment)
print(df_dummy.shape)
df_dummy.head()

### 1.4 convert object (date) to datetime format

In [ ]:
## check date format
date_cols=['ibirth','dbirth','DOB_of_Driver','Accident_Date','Accident_Time']


In [ ]:
df_dummy[date_cols].info()
"""
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1988927 entries, 0 to 1988926
Data columns (total 5 columns):
ibirth           object
dbirth           object               # object... (selected index repalce with ibirth)
DOB_of_Driver    object
Accident_Date    object
Accident_Time    object
dtypes: datetime64[ns](4), object(1)
memory usage: 91.0+ MB
"""

In [ ]:
df_dummy[date_cols].sample(10)  ## 77% missing data

In [ ]:
type(df_dummy['Accident_Time'][1000000])

In [ ]:
### fill NaN as (2020)
df= df_dummy

cols = ['Accident_Time']
for col in cols:
    print(df[col].isnull().sum())
    df[col]=df[col].fillna(str('00:00'))
    print(col, df[col].isnull().sum())

In [ ]:
type(df['Accident_Time'][1])
df['Accident_Time'].sample(10)

In [ ]:
set(df['Accident_Time'])

In [ ]:
### fill NaN as (2020)

cols = ['ibirth','dbirth','DOB_of_Driver']

for col in cols:
    print(df[col].isnull().sum())
    df[col]=df[col].fillna('09/2018')
    print(col, df[col].isnull().sum())

cols = ['Accident_Date']
for col in cols:
    print(df[col].isnull().sum())
    df[col]=df[col].fillna('2018/09')
    print(col, df[col].isnull().sum())

df.isnull().sum().sum()    
df_dummy= df

In [ ]:
df_dummy[date_cols].sample(10)

In [ ]:
df= df_dummy

date_col={'ibirth','dbirth','DOB_of_Driver'}
for col in date_col:
    df[col] =  pd.to_datetime(df[col], format='%m/%Y', errors='ignore')  #'%Y-%m-%d %H:%M:%S'   #errors='coerce'

date_col={'Accident_Date'}
for col in date_col:
    df[col] =  pd.to_datetime(df[col], format='%Y/%m',errors='ignore')  #'%Y-%m-%d %H:%M:%S'

date_col={'Accident_Time'}
for col in date_col:
    df[col] =  pd.to_datetime(df[col], format='%H:%M',errors='ignore') #'%Y-%m-%d %H:%M:%S'

df_dummy_date = df

In [ ]:
df_dummy_date[['ibirth', 'dbirth' ,'DOB_of_Driver', 'Accident_Date', 'Accident_Time']].info(verbose= True)

### 1.4.1 Extract Features from datetime data then drop datetime64 columns

In [ ]:
df_dummy_date[['ibirth', 'dbirth' ,'DOB_of_Driver', 'Accident_Date', 'Accident_Time']].loc[932405]

In [ ]:
df_dummy_date[['ibirth', 'dbirth' ,'DOB_of_Driver', 'Accident_Date', 'Accident_Time']].sample(10)

In [ ]:
## Today
today = datetime.date.today()
daysdiff = today-df_dummy_date['ibirth']
components = daysdiff.dt.components #  https://stackoverflow.com/questions/18215317/extracting-days-from-a-numpy-timedelta64-value
components.head()

In [ ]:
years = components.days/365
df_dummy_date['iYearsOld'] = years
df_dummy_date.head()

In [ ]:
today = datetime.date.today()
daysdiff = today-df_dummy_date['dbirth']
components = daysdiff.dt.components #  https://stackoverflow.com/questions/18215317/extracting-days-from-a-numpy-timedelta64-value
components.head()

In [ ]:
years = components.days/365
df_dummy_date['dYearsOld'] = years
df_dummy_date['dYearsOld'].head()

In [ ]:
today = datetime.date.today()
daysdiff = today-df_dummy_date['DOB_of_Driver']
components = daysdiff.dt.components #  https://stackoverflow.com/questions/18215317/extracting-days-from-a-numpy-timedelta64-value
components.head()
years = components.days/365
df_dummy_date['DriverYearsOld'] = years
df_dummy_date['DriverYearsOld'].head()

In [ ]:
set(df_claim['Accident_Date'])  # between 2015/01 -> 2016/12    24months

In [ ]:
daysdiff = today-df_dummy_date['Accident_Date']
components = daysdiff.dt.components #  https://stackoverflow.com/questions/18215317/extracting-days-from-a-numpy-timedelta64-value
months = components.days/30
df_dummy_date['Accident_Month_ago'] = months
df_dummy_date['Accident_Month_ago'].sample(10)

In [ ]:
from datetime import datetime, date
df_dummy_date['Acc_DayOfWeek'] = df_dummy_date.apply(lambda row: row['Accident_Date'].weekday(), axis =1)
df_dummy_date['Acc_DayOfWeek'][:5]  

In [ ]:
daysdiff = today-df_dummy_date['Accident_Time']
components = daysdiff.dt.components #  https://stackoverflow.com/questions/18215317/extracting-days-from-a-numpy-timedelta64-value
hours = components.hours
df_dummy_date['Accident_Hour'] = hours
df_dummy_date['Accident_Hour'].sample(10)

In [ ]:
df_dummy_date.head()

In [ ]:
df_dummy_date[date_cols].head()

In [ ]:
drop_date_cols = ['ibirth', 'dbirth', 'DOB_of_Driver','Accident_Date', 'Accident_Time']
df_dummy_date = df_dummy_date.drop (drop_date_cols, axis = 1)

In [ ]:
print(df_dummy_date.shape)
df_dummy_date.head()

In [ ]:
df_dummy_date.info()

### 1.5 convert object (string) to numeric

In [ ]:
# df[] = pd.to_numeric(df[], errors='coerce')

## 2. Define Train, Test, Y

In [ ]:
#df_dummy_date.isnull().sum()

In [ ]:
df = df_dummy_date 

df_train = df[df['Next_Premium']!=100]
print(df_train.shape)
df_test = df[df['Next_Premium']==100]
print(df_test.shape)
print(df_train.shape[0]+df_test.shape[0])
print(df.shape[0])

In [ ]:
df.isnull().sum().sum()

In [ ]:
y=df_train['Next_Premium']
y.head()

In [ ]:
df_train=df_train.drop(['Next_Premium'], axis=1)
print(df_train.shape)

In [ ]:
df_test=df_test.drop(['Next_Premium'], axis=1)
print(df_test.shape)

### Modelling

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [ ]:
train = df_train
test = df_test
y_train = y

In [ ]:
# ## downsampling for testing:
# train = df_train.head(50000) # 1,212,659
# y_train = y.head(50000)
# ## downsampling for testing:
# test=df_test.head(3000)   #776,268

In [ ]:
# train.to_csv('train.csv')
# test.to_csv('test.csv')
# y.to_csv('y.csv')

In [89]:
#Validation function
n_folds = 5

def mae_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    mae= np.sqrt(-cross_val_score(model, train.values, y_train.values, scoring="neg_mean_absolute_error", cv = kf))
    return(mae)

In [90]:
# Base models

In [91]:
# LGBM :
model_lgb = lgb.LGBMRegressor(objective='regression',n_estimators=200)
mae = mae_cv(model_lgb)
print("lgb mae: \n", mae)
print("lgb score: {:.4f} ({:.4f})\n".format(mae.mean(), mae.std()))

lgb mae: 
 [ 64.59504249  63.44476755  66.81633761  68.8576655   63.34448601]
lgb score: 65.4117 (2.1287)



In [97]:
model_lgb.fit(train, y)
y_pred = model_lgb.predict(test)

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [98]:
y_pred[:10]

array([  3853.44112957,   3762.3679415 ,   3762.3679415 ,   3762.3679415 ,
         4241.48189431,   3762.3679415 ,   8908.41066095,  15636.78064565,
         8846.21902987,   8649.54298545])

In [101]:
df_testing.shape

(140510, 2)

In [103]:
y_pred.shape

(776268,)

In [100]:
df_submit=df_testing
df_submit[df_testing.columns[0]]=df_testing[df_testing.columns[0]]
df_submit[df_testing.columns[1]]=y_pred
df_submit.head()

ValueError: Length of values does not match length of index

In [92]:
# # XGBoost :
# model_xgb = xgb.XGBRegressor(n_estimators=200)
# mae = mae_cv(model_xgb)
# print("Xgboost mae: \n", mae)
# print("Xgboost score: {:.4f} ({:.4f})\n".format(mae.mean(), mae.std()))

KeyboardInterrupt: 

In [ ]:
# RF :
model_rf=RandomForestRegressor(n_estimators=20)
mae = mae_cv(model_rf)
print("RandomForest mae: \n", mae)
print("\nRandomForest score: {:.4f} ({:.4f})\n".format(mae.mean(), mae.std()))